In [1]:
import argparse, os, sys, glob
import time
import torchvision.models as models
import datetime
import torch  
import torchvision.transforms as transforms  
from PIL import Image  
from vggloss import *
from omegaconf import OmegaConf
from imwatermark import WatermarkEncoder
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

from matplotlib import pyplot as plt
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from cocoimagic_utils import *


# environment
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# path
experiment_time_stamp = datetime.date.today()
config_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/configs/stable-diffusion/v1-inference.yaml"
diffusion_model_path = "/home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-5-pruned.ckpt"
# diffusion_model_path = "/home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-4-full-ema.ckpt"
img_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/gray_demo.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_11/samples/coco_00198.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_12/samples/coco_00254.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/tree_1.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/bear3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/bird.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/cat_3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zebra.jpg"
# img_path = "/home/v-penxiao/workspace/data/imagic_test/gray/landmark_image29231.jpg"

output_path = os.path.join("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs", str(datetime.date.today()).replace("-","_"))
cond_path = os.path.join(output_path, "opti_cond_xloss.pt")
unet_path = os.path.join(output_path, "opti_unet_xloss.pt")
sample_path = os.path.join(output_path, "samples")
os.makedirs(output_path, exist_ok=True)
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))
grid_count = len(os.listdir(output_path)) - 1
torch.manual_seed(0)

wm = "StableDiffusionV1"
wm_encoder = WatermarkEncoder()
wm_encoder.set_watermark('bytes', wm.encode('utf-8'))
gpu_info()

/anaconda/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'CUDA:0 (NVIDIA A100 80GB PCIe, 80995MiB)'

In [2]:
# coco = get_cocosnet().to(device)
coco = get_cocosnetv2().to(device)

config = OmegaConf.load(config_path)
model = load_model_from_config(config, diffusion_model_path)
model = model.to(device)

# vgg19_conv2_1_relu = models.vgg19(pretrained=True).features[:7].eval().to(device)
# for p in vgg19_conv2_1_relu.parameters():
#     p.requires_grad = False
# vgg19_conv3_1_relu = models.vgg19(pretrained=True).features[:11].eval().to(device)
# for p in vgg19_conv3_1_relu.parameters():
#     p.requires_grad = False
clip_model, preprocess = clip.load("ViT-B/32", device=device)
for p in clip_model.parameters():
    p.requires_grad = False

sampler = DDIMSampler(model)


if False:
    start_codenamed_parametersrandn([n_samples, 4, H // 8, W // 8], device=device)
    
precision_scope = autocast if True else nullcontext
batch_size = 1
n_rows = batch_size

latest
Network [SPADEGenerator] was created. Total number of parameters: 36.4 million. To see the architecture, do print(network).
Network [NoVGGHPMCorrespondence] was created. Total number of parameters: 9.2 million. To see the architecture, do print(network).
Load from /home/v-penxiao/workspace/cocov2/CoCosNet-v2/checkpoints/deepfashionHD/latest_net_Corr.pth
Load from /home/v-penxiao/workspace/cocov2/CoCosNet-v2/checkpoints/deepfashionHD/latest_net_G.pth
Loading model from /home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-5-pruned.ckpt
Global Step: 840000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.23.self_attn.out_proj.bias', 'vision_model.encoder.layers.8.layer_norm2.bias', 'vision_model.encoder.layers.1.self_attn.out_proj.weight', 'vision_model.encoder.layers.6.self_attn.q_proj.weight', 'vision_model.encoder.layers.20.self_attn.q_proj.bias', 'vision_model.encoder.layers.7.self_attn.k_proj.bias', 'vision_model.encoder.layers.11.mlp.fc2.bias', 'vision_model.encoder.layers.22.mlp.fc2.bias', 'vision_model.encoder.layers.17.mlp.fc1.weight', 'vision_model.encoder.layers.19.self_attn.k_proj.weight', 'vision_model.encoder.layers.5.layer_norm1.bias', 'vision_model.encoder.layers.6.self_attn.v_proj.bias', 'vision_model.encoder.layers.2.layer_norm1.bias', 'vision_model.encoder.layers.20.self_attn.k_proj.bias', 'vision_model.encoder.layers.8.mlp.fc2.bias', 'vision_model.encoder.layers.10.layer_norm2.weight', 'vision_model.encoder.layers.13.self

In [3]:
# neg_prompt = "a gray photograph of a dog, a black bench in the center, and the grassland."
neg_prompt = "A grayscale photograph of a bird standing on a branch"
# neg_prompt = "a mostly gray photograph."
prompt = "a colorful photograph of a dog sitting on a bench on the grassland."
#on a wooden bench, with the sky and the grassland"
# prompt = "a colorful photograph of a brown dog, sitting on a bench in the grassland"
# prompt = "a colorful photograph of a dog in red"
# prompt = "A colorful photograph of a red flamingo standing by the blue sea."
# prompt = "a colorful photograph a big green tree on the meadow"
# prompt = "A colorful photograph of a red flamingo standing by the sea."
# prompt = "A colorful photograph of a black bear walking in the green forest"
# prompt = "A colorful photograph of a bird standing on a branch"
# prompt = "a photograph of a sitting yellow cat"
# prompt = "a colorful photograph of a bowl of red apples"
# prompt = "a big white and red plane flying in the blue sky"
# prompt = "a red bus driving on the street"
# prompt = "a colorful photograph of a brown and white cow lies on a green grass"
# prompt = "a colorful photograph."
# prompt = "A work of art with a purple lion standing on it, a red-haired woman lying on the ground, a yellow violin falling beside her, and a yellow moon in the sky."
# prompt = "A color photograph of houses with colorful roofs and a blue sky in the background"
# prompt = "a colorful photograph of a black horse standing on a path in the countryside with a clear sky in the background."
# resize_transform = transforms.Resize(512)  
# pil_image = Image.open(img_path).convert("RGB")
# resized_image = resize_transform(pil_image)  
src_image = np.array(Image.open(img_path).convert("RGB"))

src_image = src_image.astype(np.float32) / 255.0
src_image = src_image[None].transpose(0, 3, 1, 2)
src_image = torch.from_numpy(src_image)

src_image = src_image * 2.0 - 1.0
src_image = src_image.to(device)

un_cond = model.get_learned_conditioning(batch_size * ["a grayscale photograph"])
ori_emb = model.get_learned_conditioning([prompt])
src_image_latent = model.encode_first_stage(src_image)
x_start = model.get_first_stage_encoding(src_image_latent)
if os.path.isfile("x_start.pt"):
    print("load x_start")
    start_code = torch.load("x_start.pt")
else:
    start_code = torch.randn_like(x_start)
    torch.save(start_code, "x_start.pt")

emb = ori_emb.clone()

load x_start


In [4]:
gray_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/gray_demo.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
gray = np.array(Image.open(gray_path).convert("RGB"))

gray = gray.astype(np.float32) / 255.0
gray = gray[None].transpose(0, 3, 1, 2)
gray = torch.from_numpy(gray)

gray = gray * 2.0 - 1.0
gray = gray.to(device)

In [4]:
'''
    #3. finetune diffusion model for reconstruction
'''
print("-" * 20, "finetune for reconstruction", "-" * 20)
model.model.train()
# model.model.eval()
emb.requires_grad = False
# optimizer_finetune = torch.optim.Adam([
#                                         {'params':[cond], 'lr':1e-5},
#                                         {'params':model.model.parameters(), 'lr':2e-7}], 
#                                         lr=1e-6)
# optimizer_finetune = torch.optim.Adam([cond], lr=2e-3)
optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=1e-6)
print("cond.requires_grad=",emb.requires_grad)
print("cond_copy.requires_grad=",ori_emb.requires_grad)
# model.model.eval()
# for p in model.model.parameters():
#     p.requires_grad = False
steps = 1000
criteria = torch.nn.MSELoss()
history = []

# func =  Compose([
#         Resize(256,interpolation=BICUBIC),
#         CenterCrop(224),
#         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
#     ])

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    # print(f"t_tensor = {t_tensor}")
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, emb)
    pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    ''''''
    # pred = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)

    
    pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # coco_x0_rgb = colorized(coco,src_image,pred_x0_rgb, False, output_path)
    
    # print(coco_x0_rgb.max(),coco_x0_rgb.min(),src_image.max(),src_image.min(),pred_x0_rgb.max(),pred_x0_rgb.min)
    # x_sample = 255. * rearrange(((1.+pred_x0_rgb[0])/2.).detach().cpu().numpy(), 'c h w -> h w c')
    # img = Image.fromarray(x_sample.astype(np.uint8))
    # img = put_watermark(img, wm_encoder)
    # img.save(os.path.join(sample_path, f"pred_{t_tensor.item()}.png"))
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

        # loss = criteria(pred_x0_gray_latent, x_start)
    # pixel_loss =criteria(pred_x0_rgb, src_image)
    # vgg_loss = vgg(pred_x0_rgb, src_image)
    # p_loss = vgg19_loss(src_image, pred_x0_rgb, vgg19_conv2_1_relu, vgg19_conv3_1_relu)
    # coco_clip_loss = get_clip_loss(clip_model, coco_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    pred_clip_loss = get_clip_loss(clip_model, pred_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    w = (1- t_tensor.item() / model.num_timesteps)
    # clip_loss = (0.5 * pred_clip_loss + 0.5 * coco_clip_loss)
    clip_loss = 1. * pred_clip_loss
    noise_loss = criteria(pred_noise, noise)
    # if t_int>1500:
    #     loss = .4 * clip_loss + 0.5 * noise_loss + 0.1 * p_loss
    # else:
    #     loss = .2 * clip_loss + 0.7 * noise_loss + 0.1 * p_loss
    loss = w * clip_loss + (1 - w) * noise_loss 
    # loss = clip_loss + 1. * noise_loss 
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    with torch.no_grad():
        if (t_int+1) % 100 ==0 or t_int ==0:
            print("curr_t=", t_int, "; loss=", loss.item(), f"clip loss = {clip_loss.item()}, noise loss = {noise_loss.item()}")
            ret = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
            colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)
            # colored_ret = colored_ret * 2.0 - 1.0
            # x_start =  model.get_first_stage_encoding(model.encode_first_stage(colored_ret))

# unet_for_pixel_path = "/root/stable/stable-diffusion/outputs/imagic/pixel_unet.pt"

# print("saving finetuned unet-model to path:", unet_for_pixel_path)
# torch.save(model.model.state_dict(), unet_for_pixel_path)
# print("Done\n")

cond_for_pixel_path = os.path.join(output_path,"pixel_cond.pt")

print("saving optimized_cond to path:", cond_for_pixel_path)
# torch.save(new_cond, cond_for_pixel_path)
print("Done\n")

-------------------- finetune for reconstruction --------------------
cond.requires_grad= False
cond_copy.requires_grad= False
curr_t= 0 ; loss= 0.28646349906921387 clip loss = 0.292724609375, noise loss = 0.26315397024154663
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 46 timesteps


DDIM Sampler: 100%|██████████| 46/46 [00:05<00:00,  8.28it/s]


4
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 256, 256])
torch.Size([1, 32, 512, 512])
hierarchical_scale = 0
ref shape = torch.Size([1, 3, 64, 64])
hierarchical_scale = 1
here


/anaconda/envs/ldm/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
../../cocov2/CoCosNet-v2/models/networks/ops.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = inds // width


torch.Size([1, 3, 128, 128])
hierarchical_scale = 2
hierarchical_scale = 3
input.shape = torch.Size([1, 3, 512, 512])
output.shape = torch.Size([1, 3, 512, 512])
curr_t= 99 ; loss= 0.057341188192367554 clip loss = 0.3671875, noise loss = 0.008167115040123463
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 46 timesteps


...

4
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 256, 256])
torch.Size([1, 32, 512, 512])
hierarchical_scale = 0
ref shape = torch.Size([1, 3, 64, 64])
hierarchical_scale = 1
here
torch.Size([1, 3, 128, 128])
hierarchical_scale = 2
hierarchical_scale = 3
input.shape = torch.Size([1, 3, 512, 512])
output.shape = torch.Size([1, 3, 512, 512])
curr_t= 499 ; loss= 0.175602525472641 clip loss = 0.24365234375, noise loss = 0.10006053000688553
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 46 timesteps


DDIM Sampler: 100%|██████████| 46/46 [00:05<00:00,  8.04it/s]


4
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 256, 256])
torch.Size([1, 32, 512, 512])
hierarchical_scale = 0
ref shape = torch.Size([1, 3, 64, 64])
hierarchical_scale = 1
here
torch.Size([1, 3, 128, 128])
hierarchical_scale = 2
hierarchical_scale = 3
input.shape = torch.Size([1, 3, 512, 512])
output.shape = torch.Size([1, 3, 512, 512])
curr_t= 599 ; loss= 0.0736020877957344 clip loss = 0.255615234375, noise loss = 0.017999524250626564
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 46 timesteps


DDIM Sampler: 100%|██████████| 46/46 [00:05<00:00,  9.19it/s]


4
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 256, 256])
torch.Size([1, 32, 512, 512])
hierarchical_scale = 0
ref shape = torch.Size([1, 3, 64, 64])
hierarchical_scale = 1
here
torch.Size([1, 3, 128, 128])
hierarchical_scale = 2
hierarchical_scale = 3
input.shape = torch.Size([1, 3, 512, 512])
output.shape = torch.Size([1, 3, 512, 512])


In [ ]:
for _ in range(5):
    with torch.no_grad():
        # new_cond = model.get_learned_conditioning([prompt + "blue blue blue sky"])
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
        colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)


In [ ]:
'''
    #1. embedding optimization
'''
print("-" * 20, "embedding optimization", "-" * 20)
emb.requires_grad = True
optimizer = torch.optim.Adam([emb], lr=1e-3)
criteria = torch.nn.MSELoss()
history = []
steps = 500
for t_int in range(steps):
    optimizer.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(1000, (1,), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, emb)
    # pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model(not available)'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    
    # pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))
    
    # save_image(pred_x0_rgb,"pred_rgb.jpg")
    # save_image(pred_x0_gray,"pred_gray.jpg")

    # loss = criteria(pred_x0_gray_latent, x_start)
    # loss = criteria(pred_x0, x_start)
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss = criteria(pred_noise, noise)
    loss.backward()
    history.append(loss.item())
    optimizer.step()
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item())
        with torch.no_grad():
            _ = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, 
                                   output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, 
                                   n_samples=batch_size, scale=3, ddim_steps=45)

print("saving optimized_cond to path:", cond_path)
torch.save(emb, cond_path)
print("Done\n")

In [ ]:
'''
    #2. finetune diffusion model
'''
print("-" * 20, "finetune diffusion model", "-" * 20)
# cond = torch.load(cond_path)
emb.requires_grad = False
print("cond.requires_grad=",emb.requires_grad)
# model.model.train()
model.train()
optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=1e-6)
steps = 1000
criteria = torch.nn.MSELoss()
history = []

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(model.num_timesteps, (1,), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, emb)
    # pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
      
    # pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

    # save_image(pred_x0_rgb,"pred_rgb.jpg")
    # save_image(pred_x0_gray,"pred_gray.jpg")
    
    # loss = 0.0 * criteria(pred_x0_gray_latent, x_start)  + 0. * criteria(pred_noise, noise) + 0. * vgg(pred_x0_gray, src_image)
    # loss = criteria(pred_x0_gray_latent, x_start)
    # loss = criteria(pred_x0, x_start)
    loss = criteria(pred_noise, noise)

    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item())
        with torch.no_grad():
            _ = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)

print("saving finetuned unet-model to path:", unet_path)
torch.save(model.model.state_dict(), unet_path)
print("Done\n")

In [ ]:
cond = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

In [ ]:
model.eval()
for alpha in (0.8, 0.9, 1, 1.1):
    new_c = emb * torch.Tensor([1-alpha]).to(device) + ori_emb * torch.Tensor([alpha]).to(device)
    ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [13]:
# alpha = 1.1
emb = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

model.eval()
v = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]
# zero_emb = ori_emb


# zero_emb = model.get_learned_conditioning(["A colorful photograph of a red flamingo standing by the blue sea."])
# for alpha in v:
# prompt = "a colorful photograph of a white dog sitting on a wooden black bench, with the red sky and the green grassland"
# and the green grassland
zero_emb = model.get_learned_conditioning(["a colorful photograph of a sitting brown brown brown dog"])
# zero_emb = ori_emb
    # zero_emb = model.get_learned_conditioning(["a colorful photograph of a yellow dog" + " with the black sky"])
# for alpha in v:
    # alpha = 1.7
alpha = 1.2
new_c = emb * torch.Tensor([1-alpha]).to(device) + zero_emb * torch.Tensor([alpha]).to(device)
ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)
colored_ret = colorized(coco,gray,ret.type(gray.type()).to(gray.device), True, output_path)

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:06<00:00,  8.01it/s]


4
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 256, 256])
torch.Size([1, 32, 512, 512])
hierarchical_scale = 0
ref shape = torch.Size([1, 3, 64, 64])
hierarchical_scale = 1
here
torch.Size([1, 3, 128, 128])
hierarchical_scale = 2
hierarchical_scale = 3
input.shape = torch.Size([1, 3, 512, 512])
output.shape = torch.Size([1, 3, 512, 512])


In [ ]:
from cocoimagic_utils import *
coco = get_cocosnet().to(device)

In [ ]:
model.model.eval()
cond.requires_grad = False
alpha = 0.9
n = 3
for _ in range(n):
    with torch.no_grad():
        new_c = cond * torch.Tensor([1-alpha]).to(device) + cond_copy * torch.Tensor([alpha]).to(device)
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [ ]:
model.model.eval()
cond.requires_grad = False

v = [0.8, 0.85, 0.9, 1., 1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7]
n = 1
# heights = [128 for _ in range(n)]
# widths = [128 for _ in range(len(v))]
# fig_width = n * 30  # inches
# fig_height = fig_width * sum(heights) / sum(widths)
# f, axarr = plt.subplots(len(v),n, figsize=(fig_width, fig_height),
#                         gridspec_kw={'height_ratios':heights})
for id, alpha in enumerate(v):
    new_c = cond * torch.Tensor([1-alpha]).to(device) + cond_copy * torch.Tensor([alpha]).to(device)
    for j in range(n):
        with torch.no_grad():
            ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [ ]:
emb = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

In [ ]:
'''
    #3. finetune diffusion model for reconstruction
'''
print("-" * 20, "finetune for reconstruction", "-" * 20)
model.model.train()
optimizer_finetune = torch.optim.Adam([
                                        {'params':[cond], 'lr':1e-5},
                                        {'params':model.model.parameters(), 'lr':5e-7}], 
                                        lr=1e-6)
# optimizer_finetune = torch.optim.Adam([cond], lr=2e-3)
# optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=5e-8)
cond.requires_grad = True
print("cond.requires_grad=",cond.requires_grad)
# model.model.eval()
# for p in model.model.parameters():
#     p.requires_grad = False
steps = 3000
criteria = torch.nn.MSELoss()
history = []

# func =  Compose([
#         Resize(256,interpolation=BICUBIC),
#         CenterCrop(224),
#         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
#     ])

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, cond)
    pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    
    pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    coco_x0_rgb = colorized(coco,src_image,pred_x0_rgb, False, output_path)
    
        # print(coco_x0_rgb.max(),coco_x0_rgb.min(),src_image.max(),src_image.min(),pred_x0_rgb.max(),pred_x0_rgb.min)
        # x_sample = 255. * rearrange(((1.+pred_x0_rgb[0])/2.).detach().cpu().numpy(), 'c h w -> h w c')
        # img = Image.fromarray(x_sample.astype(np.uint8))
        # img = put_watermark(img, wm_encoder)
        # img.save(os.path.join(sample_path, f"coco_{t_int}.png"))
        # pred_x0_gray = degration(pred_x0_rgb)
        # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

        # loss = criteria(pred_x0_gray_latent, x_start)
    # pixel_loss =criteria(pred_x0_rgb, src_image)
    # vgg_loss = vgg(pred_x0_rgb, src_image)
    # p_loss = vgg19_loss(src_image, pred_x0_rgb, vgg19_conv2_1_relu,vgg19_conv3_1_relu)
    coco_clip_loss = get_clip_loss(clip_model, coco_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    pred_clip_loss = get_clip_loss(clip_model, pred_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    clip_loss = 1.0 * coco_clip_loss + 1. * pred_clip_loss
    noise_loss = criteria(pred_noise, noise)
    if t_int>2500:
        loss = 1 * clip_loss + 0.1 * noise_loss 
    else:
        loss = 0.2 * clip_loss + 1. * noise_loss 
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item(), f"clip loss = {clip_loss.item()}, noise loss = {noise_loss.item()}")
        # new_cond = new_cond * torch.Tensor([1-0.1]).to(device) + model.get_learned_conditioning([prompt]) * torch.Tensor([0.1]).to(device)
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
        colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)

# unet_for_pixel_path = "/root/stable/stable-diffusion/outputs/imagic/pixel_unet.pt"

# print("saving finetuned unet-model to path:", unet_for_pixel_path)
# torch.save(model.model.state_dict(), unet_for_pixel_path)
# print("Done\n")

cond_for_pixel_path = os.path.join(output_path,"pixel_cond.pt")

print("saving optimized_cond to path:", cond_for_pixel_path)
# torch.save(new_cond, cond_for_pixel_path)
print("Done\n")

In [ ]:
src_image = np.array(Image.open("/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png").convert("RGB"))
src_image = src_image.astype(np.float32) / 255.0
src_image = src_image[None].transpose(0, 3, 1, 2)
src_image = torch.from_numpy(src_image)
src_image = src_image * 2.0 - 1.0
src_image = src_image.to(device)
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    
func =  Compose([
        Resize(256,interpolation=BICUBIC),
        CenterCrop(224),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

# image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
src_image = (src_image+1.)/2.
image = func(src_image)
image.shape

In [ ]:
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
neg_prompt = "a grayscale photo"
# neg_prompt = "a mostly gray photograph."
prompt = "a colorful photo"
# prompt = "a colorful photograph."
# image = preprocess(Image.open("/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png")).unsqueeze(0).to(device)
#/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/gray_demo.png
image = preprocess(Image.open("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_03/samples/00252.png")).unsqueeze(0).to(device)
i_e = clip_model.encode_image(image)
p_t_e = clip_model.encode_text(clip.tokenize(prompt).to(image.device))
n_t_e = clip_model.encode_text(clip.tokenize(neg_prompt).to(image.device))
g_t_e = clip_model.encode_text(clip.tokenize("grayscale").to(image.device))
# gray_t_e = model.encode_text(clip.tokenize("a gray photograph").to(image.device))
# color_t_e = model.encode_text(clip.tokenize("a colorful photograph").to(image.device))
p_cos_sim = torch.nn.functional.cosine_similarity(i_e, p_t_e).mean()
n_cos_sim = torch.nn.functional.cosine_similarity(i_e, n_t_e).mean()
g_cos_sim = torch.nn.functional.cosine_similarity(i_e, g_t_e).mean()
print(p_cos_sim)
print(n_cos_sim)
print(g_cos_sim)

In [ ]:
n_cos_sim

In [ ]:
torch.nn.functional.cosine_similarity(i_e, n_t_e).mean()

In [ ]:
torch.nn.functional.cosine_similarity(i_e, p_t_e).mean()

In [ ]:
clip_model.encode_text(clip.tokenize([prompt,neg_prompt]).to(device)).shape

In [ ]:
cond.shape

In [ ]:
clip.tokenize(neg_prompt).shape